### 검색도구: Tavily Search

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# TavilySearchResults 클래스를 langchain_community.tools.tavily_search 모듈에서 가져옵니다.
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=5은 검색 결과를 5개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=3)

In [4]:
# 검색 결과를 가져옵니다.
search.invoke("판교 제로원에이아이 전화번호는 무엇인가요?")

[{'url': 'https://news.mt.co.kr/mtview.php?no=2023110910333335010',
  'content': '제로원에이아이 (대표 송준호)는 판교 본사를 방문한 사우디아라비아 압둘라지즈 압둘라흐 알리 알 다그리르 이스트 (Abdullaziz Abdullah Ali Al Daghrir Est.) 회사의 알둘라지즈 (Abdullaziz) 대표이사와 상호업무협약 양해각서 (MOU)를 맺었다고 밝혔다.'},
 {'url': 'https://www.pangyotechnovalley.org/base/board/read?boardManagementNo=16&boardNo=590&menuLevel=2&menuNo=15',
  'content': "제로원에이아이-서울대병원 온라인 의료 AI 경진대회 관련 이미지 | 제공-제로원에이아이. 의료 인공지능 플랫폼 기술선도 기업 '㈜제로원에이아이(대표 송준호)'가 최근 진행된 서울대학교병원 온라인 의료 인공지능 경진대회에 '의료 보안 데이터 특화 인공지능 분석환경(AI SaaS Solution)'을 제공해 ..."},
 {'url': 'https://www.etnews.com/20211005000250',
  'content': '보안 데이터 인공지능(AI) 플랫폼 기술 선도 기업 제로원에이아이(대표 송준호)는 지난 8월 30일부터 2주간 열렸던 온라인 AI 경진대회 판교 AI 캠프 ...'},
 {'url': 'https://v.daum.net/v/20211005150704539',
  'content': "보안 데이터 인공지능(AI) 플랫폼 기술 선도 기업 제로원에이아이(대표 송준호)는 지난 8월 30일부터 2주간 열렸던 온라인 AI 경진대회 '판교 AI 캠프' 최종결선에서 우수상을 수상했다고 5일 밝혔다. 경기도가 주최하고 경기도경제과학진흥원에서 주관한 이번 행사는 판교테크노밸리를 중심으로 AI 기업 ..."},
 {'url': 'https://www.aitimes.kr/news/articl

### Agent 가 사용할 도구 목록 정의

In [5]:
# tools 리스트에 search를 추가합니다.
tools = [search]

### Agent with smaller model

In [6]:
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_messages
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools.render import render_text_description_and_args
from langchain_core.messages import AIMessage, HumanMessage

from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import tool


local_llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    model="cognitivecomputations/dolphin-2.9-llama3-8b-gguf",
    temperature=0,
)


@tool
def search_phonenumber(query: str) -> str:
    """장소에 대한 전화번호 검색 결과를 반환할 때 사용되는 도구입니다"""
    return "판교 제로원에이아이 전화번호: 010-1234-5678\n\n서울 OOO 전화번호: 02-123-4567"


tools = [search_phonenumber]

In [7]:
from typing import Tuple, List

chat_model_with_stop = local_llm.bind(
    stop=["Observation", "\nObservation", "\n관측"])

# Inspiration taken from hub.pull("hwchase17/react-json")
system_message = f"""Answer the following questions as best you can.
You can answer directly if the user is greeting you or similar.
Otherise, you have access to the following tools:

{render_text_description_and_args(tools).replace('{', '{{').replace('}', '}}')}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use)
and a `action_input` key (with the input to the tool going here).
The only values that should be in the "action" field are: {[t.name for t in tools]}
The $JSON_BLOB should only contain a SINGLE action, 
do NOT return a list of multiple actions.
Here is an example of a valid $JSON_BLOB:
```
{{{{
    "action": $TOOL_NAME,
    "action_input": $INPUT
}}}}
```
The $JSON_BLOB must always be enclosed with triple backticks!

ALWAYS use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action:```
$JSON_BLOB
```
Observation: the result of the action... 
(this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Reminder to always use the exact characters `Final Answer` when responding.'
"""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            system_message,
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)


def _format_chat_history(chat_history: List[Tuple[str, str]]):
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [8]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_messages(x["intermediate_steps"]),
        "chat_history": lambda x: (
            _format_chat_history(x["chat_history"]) if x.get(
                "chat_history") else []
        ),
    }
    | prompt
    | chat_model_with_stop
    | ReActJsonSingleInputOutputParser()
)


# Add typing for input
class AgentInput(BaseModel):
    input: str
    chat_history: List[Tuple[str, str]] = Field(
        ..., extra={"widget": {"type": "chat", "input": "input", "output": "output"}}
    )


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True).with_types(
    input_type=AgentInput
)

In [10]:
# 검색 결과를 요청 후 질문에 대한 답변을 출력합니다.
response = agent_executor.invoke(
    {
        "input": "판교 제로원에이아이 전화번호를 웹 검색하여 결과를 알려주세요.",
        "chat_history": [],
    }
)
print(f'답변: {response["output"]}')
# API 연결 문제로 실행 실패

In [12]:
parser = ReActJsonSingleInputOutputParser()
action = parser.parse(response["output"])

In [14]:
# !pip install langgraph

   ---------------------------------------- 0.0/102.6 kB ? eta -:--:--
   --- ------------------------------------ 10.2/102.6 kB ? eta -:--:--
   ---------------------------------------- 102.6/102.6 kB 1.5 MB/s eta 0:00:00


In [15]:
from langgraph.prebuilt import ToolExecutor

# ToolExecutor 클래스의 인스턴스를 생성
tool_executor = ToolExecutor(tools)

In [17]:
# 결과 확인
tool_executor.invoke(action)

### 검색 결과를 바탕으로 RAG

In [18]:
from langchain_core.agents import AgentAction
from langchain_core.prompts import ChatPromptTemplate

from langchain_community.chat_models import ChatOllama
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [19]:
eeve = ChatOllama(
    model="EEVE-Korean-10.8B:long",
    temperature=0,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

In [22]:
# 결과 확인
dummy_tool_executor = ToolExecutor([search_phonenumber])
dummy_result = dummy_tool_executor.invoke(action)
dummy_result

In [23]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful AI Assistant. You must answer the question based on the context.\n#Context: {context}",
        ),
        ("user", "{question}"),
    ]
)

chain = prompt | eeve

In [27]:
_ = chain.invoke(
    {
        "question": "판교 제로원에이아이 전화번호를 웹 검색하여 결과를 알려주세요.",
        "context": dummy_result,
    }
)

In [31]:
search_action = AgentAction(
    log="test",
    tool=TavilySearchResults().name,
    tool_input={"query": "판교 제로원에이아이 전화번호"},
)

In [32]:
tool_executor = ToolExecutor([TavilySearchResults(k=3)])
result = tool_executor.invoke(search_action)
result

[{'url': 'https://www.saramin.co.kr/zf_user/jobs/view?rec_idx=40916614',
  'content': '(주)제로원에이아이에서 채용공고가 시작되면 ... ㆍ 근무지역: (13449) 경기 성남시 수정구 대왕판교로 815 판교창조경제밸리 기업지원허브 p842(시흥동) 유연근무제(탄력적근로시간제, 재량근로시간제, 보상휴가제) 및 포괄임금제 운영 ... 전화번호, 이메일 주소 ...'},
 {'url': 'https://www.pangyotechnovalley.org/base/board/read?boardManagementNo=16&boardNo=674&menuLevel=2&menuNo=15',
  'content': '판교 테크노밸리 ... 보안 데이터 인공지능 플랫폼 기술 선도 기업 ㈜제로원에이아이(대표 송준호)는 koren ip 백본망 트래픽 감시 및 장애 분석을 위한 ai 관제 시스템 구축 사업의 8종 기능에 대해 한국전자통신연구원(etri)의 시험 통과 인증을 획득했다고 12일 ...'},
 {'url': 'https://www.pangyotechnovalley.org/base/board/read?boardManagementNo=16&boardNo=510&menuLevel=2&menuNo=13',
  'content': "판교 창업존, 제7회 스타트업 815 'd.n.a분야' ir 성료 (2021.10.29) ... '제로원에이아이'(ai 플랫폼(saas), 빅데이터 플랫폼(paas)) '스마트인사이드에이아이'(건설 현장 특화 영상 ai) '옐로나이프'(차량용 디지털 계기판 ui 커스터마이징 기술) '오터컴퍼니'(키워드 기반 맞춤 ..."},
 {'url': 'https://www.jumpit.co.kr/company/5018?company_nm=제로원에이아이',
  'content': '제로원에이아이. ... 업력 6년차 (2019년 설립)대표주소 경기 성남시 수정구 창업로 43 판교글로벌비즈센터 b70

In [33]:
search_result = " ".join([r["content"] for r in result])
search_result

"(주)제로원에이아이에서 채용공고가 시작되면 ... ㆍ 근무지역: (13449) 경기 성남시 수정구 대왕판교로 815 판교창조경제밸리 기업지원허브 p842(시흥동) 유연근무제(탄력적근로시간제, 재량근로시간제, 보상휴가제) 및 포괄임금제 운영 ... 전화번호, 이메일 주소 ... 판교 테크노밸리 ... 보안 데이터 인공지능 플랫폼 기술 선도 기업 ㈜제로원에이아이(대표 송준호)는 koren ip 백본망 트래픽 감시 및 장애 분석을 위한 ai 관제 시스템 구축 사업의 8종 기능에 대해 한국전자통신연구원(etri)의 시험 통과 인증을 획득했다고 12일 ... 판교 창업존, 제7회 스타트업 815 'd.n.a분야' ir 성료 (2021.10.29) ... '제로원에이아이'(ai 플랫폼(saas), 빅데이터 플랫폼(paas)) '스마트인사이드에이아이'(건설 현장 특화 영상 ai) '옐로나이프'(차량용 디지털 계기판 ui 커스터마이징 기술) '오터컴퍼니'(키워드 기반 맞춤 ... 제로원에이아이. ... 업력 6년차 (2019년 설립)대표주소 경기 성남시 수정구 창업로 43 판교글로벌비즈센터 b705. 보안 데이터 인공지능 (AI) 플랫폼 기술 선도 기업 제로원에이아이 (대표 송준호)는 지난 8월 30일부터 2주간 열렸던 온라인 AI 경진대회 '판교 AI 캠프 ..."

In [35]:
_ = chain.invoke(
    {
        "question": "판교 제로원에이아이 전화번호를 웹 검색하여 결과를 알려주세요.",
        "context": search_result,
    }
)